## Small-to-big Retrieval-Augmented Generation 

Keith Ballinger and Megan O'Keefe, 2024

Small-to-big RAG is a form of recursive RAG, where small chunks are connected to larger chunks (eg. a passage of a document is connected to the full document). Initially, only small chunks are retrieved during RAG. If needed, the model can request additional context, at which point the larger context can be added.

Small-to-big RAG is a way to work around the limits of dense, meaning-rich vector embeddings, where passing in too much context can result in a loss of meeting in the vector representation.  
(Text-embedding-gecko has a text input limit of 3,072 tokens)  

Small-to-big is also a good option if you are working with long or complex documents that require context exceeding the ideal chunk size. 

Lastly, Small-to-big can help you optimize inference costs, especially if you're paying per token (eg. Gemini API on Vertex AI) — because if a small amount of input context is enough for the model to generate an answer, you don't have to provide the whole large document as context. Said another way - you're only paying for the number of context tokens you actually need to generate an accurate response from Gemini.

In this example, we'll walk through a Small-to-big RAG example using a GitHub codebase called Online Boutique. This is a microservices, polyglot sample application. We'll help a new Online Boutique user (or contributor) navigate this large codebase by providing a Q&A chatbot functionality, grounded on the codebase. 


![](architecture.png)

Prerequisites:
- Google Cloud account
- Google Cloud project with billing enabled 
- Enable the Vertex AI API

This notebook uses the following products and tools:
- Vertex AI - Gemini API 
- Vertex AI - Text Embeddings API 
- Chroma (in-memory vector database)  




### Setup 

In [24]:
EMBEDDING_MODEL="textembedding-gecko@003"
GENERATIVE_MODEL="gemini-1.0-pro"
PROJECT_ID="mokeefe-genai-test"
REGION="us-central1"

In [4]:
! pip install "google-cloud-aiplatform>=1.38"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 23.8 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.33.1
    Uninstalling google-cloud-aiplatform-1.33.1:
      Successfully uninstalled google-cloud-aiplatform-1.33.1

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.10 -m pip install --upgrade pip


In [28]:
from google.cloud import aiplatform
import vertexai
from vertexai.language_models import TextEmbeddingModel
from vertexai.generative_models import GenerativeModel, ChatSession
import os 


In [18]:
def get_text_embedding(doc) -> list:
    model = TextEmbeddingModel.from_pretrained(EMBEDDING_MODEL)
    embeddings = model.get_embeddings([doc])
    if len(embeddings) > 1:
        raise ValueError("More than one embedding returned.")
    if len(embeddings) == 0:
        raise ValueError("No embedding returned.")
    return embeddings[0].values

In [43]:
vertexai.init(project=PROJECT_ID, location=REGION)
model = GenerativeModel(GENERATIVE_MODEL)
chat = model.start_chat()

def gemini_inference(chat: ChatSession, prompt: str) -> str:
    text_response = []
    responses = chat.send_message(prompt, stream=False)
    for chunk in responses:
        text_response.append(chunk.text)
    return "".join(text_response)

In [45]:
code = """
func init() {
	log = logrus.New()
	log.Level = logrus.DebugLevel
	log.Formatter = &logrus.JSONFormatter{
		FieldMap: logrus.FieldMap{
			logrus.FieldKeyTime:  "timestamp",
			logrus.FieldKeyLevel: "severity",
			logrus.FieldKeyMsg:   "message",
		},
		TimestampFormat: time.RFC3339Nano,
	}
	log.Out = os.Stdout
"""
prompt = """
Summarize the following code: 
{}
""".format(code)
result = gemini_inference(chat, prompt)
print(result)

This code sets up a logging mechanism using the Logrus library. It configures the logging level to debug, uses a custom JSONFormatter with specific field names and timestamps, and sets the logging output to the standard output stream.


### Get summaries of code files 

First, we'll use Gemini on Vertex AI to get short summaries of each code file. 

In [46]:
# for every file in onlineboutique-codefiles/, read it in, and get a summary 
summaries = {} 
for file in os.listdir("onlineboutique-codefiles/"):
    temp = {}
    with open("onlineboutique-codefiles/" + file, "r") as f:
        print("Processing file: ", file)
        content = f.read()     
        temp["content"] = content 
        prompt = """ 
        You are a helpful code summarizer. Here is a source code file. Please identify the programming language and summarize it in three sentences or less. Give as much detail as possible, including function names and libraries used. Code: 
        {}
        """.format(content)
        try: 
            summary = gemini_inference(chat, prompt)
            temp["summary"] = summary
            summaries[file] = temp
        except Exception as e:
            print("Error processing file: ", file)
            print(e)

Processing file:  checkoutservice.go
Processing file:  shippingservice.go
Processing file:  adservice.java
Processing file:  paymentservice.js
Processing file:  recommendationservice.py
Processing file:  emailservice.py
Processing file:  frontend.go
Processing file:  productcatalog.go
Processing file:  cartservice.cs
Processing file:  currencyservice.js


In [47]:
print(summaries)

{'checkoutservice.go': {'content': '// Copyright 2018 Google LLC\n//\n// Licensed under the Apache License, Version 2.0 (the "License");\n// you may not use this file except in compliance with the License.\n// You may obtain a copy of the License at\n//\n//      http://www.apache.org/licenses/LICENSE-2.0\n//\n// Unless required by applicable law or agreed to in writing, software\n// distributed under the License is distributed on an "AS IS" BASIS,\n// WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n// See the License for the specific language governing permissions and\n// limitations under the License.\n\npackage main\n\nimport (\n\t"context"\n\t"fmt"\n\t"net"\n\t"os"\n\t"time"\n\n\t"cloud.google.com/go/profiler"\n\t"github.com/google/uuid"\n\t"github.com/pkg/errors"\n\t"github.com/sirupsen/logrus"\n\t"google.golang.org/grpc"\n\t"google.golang.org/grpc/codes"\n\t"google.golang.org/grpc/status"\n\n\tpb "github.com/GoogleCloudPlatform/microservices-demo/src/chec

In [72]:
import pandas as pd
df =  pd.DataFrame.from_dict(summaries, orient='index')

In [73]:
df.head()

,content,summary
checkoutservice.go,// Copyright 2018 Google LLC\n//\n// Licensed ...,This Go program implements an order checkout m...
shippingservice.go,// Copyright 2018 Google LLC\n//\n// Licensed ...,This Go program implements a shipping service ...
adservice.java,"/*\n * Copyright 2018, Google LLC.\n *\n * Lic...",This Java program implements an ad service usi...
paymentservice.js,// Copyright 2018 Google LLC\n//\n// Licensed ...,This JavaScript code defines a class `HipsterS...
recommendationservice.py,#!/usr/bin/python\n#\n# Copyright 2018 Google ...,This Python code implements a gRPC server for ...


In [74]:
# the first column should be named "filename"
df = df.reset_index()
df = df.rename(columns = {'index':'filename'})
df.head()

,filename,content,summary
0,checkoutservice.go,// Copyright 2018 Google LLC\n//\n// Licensed ...,This Go program implements an order checkout m...
1,shippingservice.go,// Copyright 2018 Google LLC\n//\n// Licensed ...,This Go program implements a shipping service ...
2,adservice.java,"/*\n * Copyright 2018, Google LLC.\n *\n * Lic...",This Java program implements an ad service usi...
3,paymentservice.js,// Copyright 2018 Google LLC\n//\n// Licensed ...,This JavaScript code defines a class `HipsterS...
4,recommendationservice.py,#!/usr/bin/python\n#\n# Copyright 2018 Google ...,This Python code implements a gRPC server for ...


In [76]:
# write to csv
df.to_csv("code_summaries.csv", index=False)

### Convert summaries to embeddings

In [77]:
! pip install chromadb



  Using cached chroma_hnswlib-0.7.3-cp310-cp310-macosx_11_0_arm64.whl.metadata (252 bytes)
  Using cached pulsar_client-3.4.0-cp310-cp310-macosx_10_15_universal2.whl.metadata (1.0 kB)
  Using cached tokenizers-0.15.2-cp310-cp310-macosx_11_0_arm64.whl.metadata (6.7 kB)
  Using cached PyPika-0.48.9-py2.py3-none-any.whl
  Using cached overrides-7.7.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached bcrypt-4.1.2-cp39-abi3-macosx_10_12_universal2.whl.metadata (9.5 kB)
  Using cached kubernetes-29.0.0-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached mmh3-4.1.0-cp310-cp310-macosx_11_0_arm64.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.4 MB/s eta 0:00:00
  Using cached starlette-0.36.3-py3-none-any.whl.metadata (5.9 kB)
  Using cached websocket_client-1.7.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached httptools-0.6.1-cp310-cp310-macosx_10_9_universal2.whl.metadata (3.6 kB)
  Using cac

In [115]:
import chromadb
chroma_client = chromadb.Client()

In [117]:
collection = chroma_client.create_collection(name="code_summaries2")



In [124]:
df.head()

,filename,content,summary
0,checkoutservice.go,// Copyright 2018 Google LLC\n//\n// Licensed ...,This Go program implements an order checkout m...
1,shippingservice.go,// Copyright 2018 Google LLC\n//\n// Licensed ...,This Go program implements a shipping service ...
2,adservice.java,"/*\n * Copyright 2018, Google LLC.\n *\n * Lic...",This Java program implements an ad service usi...
3,paymentservice.js,// Copyright 2018 Google LLC\n//\n// Licensed ...,This JavaScript code defines a class `HipsterS...
4,recommendationservice.py,#!/usr/bin/python\n#\n# Copyright 2018 Google ...,This Python code implements a gRPC server for ...


In [125]:

# iterate over dataframe. convert summary into embeddings. insert summary into collection. 
for index, row in df.iterrows():
    fn = row["filename"]
    print("Getting embedding for: ", fn)
    summ = row["summary"] 
    print(summ)
    e = get_text_embedding(summ) 
    print(e)
    # add vector embedding to in-memory Chroma database. 
    # the "small" summary embedding is linked to the "big" raw code file through the metadata key, "filename." 
    collection.add(
        embeddings=[e],
        documents=[summ],
        metadatas=[{"filename": fn}],
        ids=[fn])
    

Getting embedding for:  checkoutservice.go
This Go program implements an order checkout microservice that uses gRPC and OpenTelemetry for tracing and logging. It demonstrates how a client can interact with multiple dependent services, such as product catalog, cart management, shipping, currency conversion, payment, and email services, to complete a checkout process for a user. The code includes functions for:
- Initializing the microservice with logging, OpenTelemetry tracing, and profiler (if enabled).

- Setting up clients for gRPC connections to other dependent services using secure connections and tracing interceptors.

- Implementing the checkout flow, which involves preparing and validating order details, charging the customer's credit card for payment, shipping the order, sending an order confirmation via email, and finally returning the order details and tracking information to the client.

- Handling interactions with dependent services, such as obtaining a shipping quote, con

Insert of existing embedding ID: checkoutservice.go
Add of existing embedding ID: checkoutservice.go


[0.07725605368614197, -0.04385218024253845, -0.04713913053274155, -0.023020315915346146, 0.07190258800983429, 0.016274606809020042, 0.034076329320669174, -0.030801227316260338, -0.01800406165421009, 0.015079783275723457, 0.008629266172647476, -0.00517351133748889, -0.03162432461977005, -0.05305308848619461, 0.06464062631130219, -0.07969357818365097, 0.011020965874195099, -0.0026579967234283686, -0.017125437036156654, 0.0043101683259010315, -0.023320090025663376, 0.0069647859781980515, -0.002909911796450615, -0.07606586813926697, -0.004090012516826391, -0.01195779349654913, -0.013160943984985352, -0.011458481661975384, -0.04560025408864021, 0.023190302774310112, -0.08753946423530579, 0.017151180654764175, -0.040744662284851074, 0.034861236810684204, -0.02743864804506302, -0.032253652811050415, -0.014250040985643864, -0.05139617994427681, -0.016670359298586845, 0.008299749344587326, 0.04472150653600693, -0.03625822439789772, 0.009687528014183044, -0.04815075173974037, -0.0107560278847813

Insert of existing embedding ID: shippingservice.go
Add of existing embedding ID: shippingservice.go


[0.048276387155056, -0.062226325273513794, -0.05805008113384247, -0.020354412496089935, 0.03065471723675728, -0.0008281287155114114, 0.016136495396494865, -0.04053480550646782, -0.00028891387046314776, 0.02445978857576847, 0.018503056839108467, -0.030242295935750008, -0.025586189702153206, -0.03699909523129463, 0.05606578662991524, -0.07571646571159363, 0.02837836928665638, -0.021757591515779495, 0.0068412600085139275, 0.005887643434107304, -0.0013778163120150566, -0.022803623229265213, -0.009180468507111073, -0.049686651676893234, -0.002797492081299424, -0.005241040606051683, -0.007122145965695381, -0.01379387266933918, -0.034944649785757065, 0.03735152631998062, -0.08312006294727325, 0.016045337542891502, -0.05456896126270294, 0.024009166285395622, -0.011958598159253597, -0.04820960387587547, -0.0027152597904205322, -0.02896544337272644, -0.02996102347970009, 0.006323061417788267, 0.044962141662836075, -0.02777842804789543, 0.008037594147026539, -0.011564855463802814, -0.013423383235

...

[0.05909634381532669, -0.04795029014348984, -0.0779849961400032, -0.0210283100605011, 0.03512101620435715, -0.0448787659406662, 0.05931307375431061, -0.043263375759124756, 0.028117714449763298, 0.022826338186860085, 0.004587385803461075, -0.011379645206034184, 0.018302517011761665, -0.030121976509690285, 0.03687430918216705, -0.07729227095842361, 0.023681191727519035, 0.020648593083024025, 0.03337412327528, -0.02020190842449665, 0.005892492830753326, 0.003674866631627083, -0.03446590527892113, -0.0565999411046505, 0.011625253595411777, -0.01786244846880436, -0.015261686407029629, 0.004656489472836256, -0.040256548672914505, -0.007298322860151529, -0.07361028343439102, 0.017334669828414917, -0.05703378841280937, 0.025901250541210175, -0.005665106233209372, -0.05071297287940979, -0.0050275083631277084, -0.01992986351251602, 0.0002344283420825377, 0.03814275190234184, 0.004230056423693895, 0.010929610580205917, 0.001305771991610527, 0.011971917934715748, -0.007732757367193699, -0.03091937

Insert of existing embedding ID: cartservice.cs
Add of existing embedding ID: cartservice.cs


[0.013436024077236652, -0.033740874379873276, -0.07724981009960175, -0.022609591484069824, 0.021359886974096298, 0.011365088634192944, 0.06243870407342911, -0.04057154804468155, 0.03864677622914314, 0.007222859188914299, 0.019559668377041817, -0.0031341686844825745, -0.029376782476902008, 0.005596999078989029, 0.04253990948200226, -0.05200964957475662, 0.04256049171090126, -0.04782113805413246, -0.019434761255979538, -0.016071008518338203, -0.022707166150212288, -0.007867855951189995, -0.0508047379553318, -0.022275306284427643, -0.0029705455526709557, -0.009238731116056442, 0.010551135055720806, -0.03050096519291401, -0.015504221431910992, -0.0178045816719532, -0.10649191588163376, 0.018848851323127747, -0.05778229236602783, 0.01336393691599369, -0.04281698167324066, -0.05233284458518028, -0.021308962255716324, 0.01001199521124363, -0.029879335314035416, 0.013535217382013798, 0.008819728158414364, -0.02639913186430931, -0.00047470134450122714, -0.03273341804742813, 0.003859217511489987

Insert of existing embedding ID: currencyservice.js
Add of existing embedding ID: currencyservice.js


[0.037494760006666183, -0.043416935950517654, -0.036206211894750595, 0.015522376634180546, 0.0712287500500679, 0.01265436876565218, 0.040616631507873535, -0.022079821676015854, 0.04669800028204918, 0.04457077011466026, 0.022742727771401405, -0.01604347862303257, -0.03297552466392517, -0.05915484204888344, 0.056912731379270554, -0.06128446012735367, 0.0366661511361599, -0.0038233078084886074, -0.002996894996613264, -0.023889536038041115, -0.039682786911726, -0.03839285299181938, 0.029229428619146347, -0.043411921709775925, 0.007851568050682545, -0.02927626110613346, 0.019319383427500725, -0.023972876369953156, -0.03372568637132645, -0.002181079937145114, -0.06707143038511276, 0.0356481596827507, -0.04747087135910988, 0.04443633183836937, -0.032780542969703674, -0.036870114505290985, 0.011805243790149689, 0.014913422986865044, 0.012086490169167519, -0.012550326995551586, 0.027388235554099083, -0.027105584740638733, -0.0008886220748536289, -0.027693239971995354, -0.0054422421380877495, -0

In [127]:
# test-query 
query = "How does the ad service work?"
query_emb = get_text_embedding(query)
result = collection.query(
    query_embeddings=[query_emb], 
    n_results=3
)
print(result)

{'ids': [['adservice.java', 'recommendationservice.py', 'currencyservice.js']], 'distances': [[0.6757434606552124, 0.8429567217826843, 0.8514495491981506]], 'metadatas': [[{'filename': 'adservice.java'}, {'filename': 'recommendationservice.py'}, {'filename': 'currencyservice.js'}]], 'embeddings': None, 'documents': [['This JavaScript code sets up a gRPC server for a currency conversion service using the `@grpc/grpc-js` library. It includes functions for:\n- Initializing OpenTelemetry tracing (if enabled) and Stackdriver Profiler (if enabled).\n- Loading protocol buffer definitions and adding corresponding service definitions to the gRPC server.\n- Handling requests for supported currencies and currency conversions.\n- Handling health checks.\n- Starting the gRPC server on a specified port.', 'This JavaScript code sets up a gRPC server for a currency conversion service using the `@grpc/grpc-js` library. It includes functions for:\n- Initializing OpenTelemetry tracing (if enabled) and St

### Complete the Small-to-big RAG workflow 

In [130]:
def small_to_big(user_prompt):
    # SMALL: first, run RAG with the summary embeddings to try to get a response
    query_emb = get_text_embedding(user_prompt)
    result = collection.query(
        query_embeddings=[query_emb], 
        n_results=1
    )
        
    # process results into something we can pass to Gemini 
    processed_result = {}
    d = result["documents"][0]
    for i in range(0, len(d)):
        summary = d[i]
        filename = result["metadatas"][0][i]["filename"]
        processed_result[filename] = summary
  
    prompt_with_small = """
    You are a codebase helper. You will be given a user's question about the codebase, along with 
    summaries of relevant code files. Do your best to generate an answer with the short summaries.  
    If you feel confident in your answer, respond with "The answer is:" followed by your answer.
    If you need more information, respond with "Need full context," and you will be prompted again with the 
    full relevant code file. 

    The user query is: {} 

    The summaries are: {}
    """.format(user_prompt, str(processed_result))
    print(prompt_with_small)
    small_result = gemini_inference(chat, prompt_with_small) 
    print("🐝 SMALL RESULT: " + small_result)
    # we're done if Gemini is confident with just the summaries as context... 
    if "need full context" not in small_result.lower():
        return "Gemini responded with just the small summaries as context: " + small_result + " 🎉"
        
    # otherwise, move on to BIG: 
    # IF we need the full context, get the filename that most closely matches the user's question
    prompt_to_get_filename = """ 
    You are a codebase helper. The list of code files that you know about: 
    - checkoutservice.go
    - shippingservice.go
    - adservice.java
    - paymentservice.js
    - recommendationservice.py
    - emailservice.py
    - frontend.go
    - productcatalog.go
    - cartservice.cs
    - currencyservice.js

    The user asks the following question about the codebase: {}

    Please respond with the filename that most closely matches the user's question. Respond with ONLY the filename. 
    """.format(user_prompt)
    filename = gemini_inference(chat, prompt_to_get_filename)
    print("Prompted for a specific filename, Gemini said: " + filename)
    # is the filename in the dataframe? 
    if filename not in df["filename"].values:
        return "⚠️ Error: filename {} not found in dataframe".format(filename)
    
    # get the full code file 
    full_code = df[df["filename"] == filename]["content"].values[0]
    prompt_with_big = """ 
    You are a codebase helper. You will be given a user's question about the codebase, along with a complete source code file. Respond to the user's question with as much detail as possible.

    The user query is: {}
    
    The full code file is: {}
    """.format(user_prompt, full_code) 

    return gemini_inference(chat, prompt_with_big)

### Test it out 

In [129]:
# an example of a query where only the small (summary) step is needed
small_to_big("How does the ad service work?")


    You are a codebase helper. You will be given a user's question about the codebase, along with 
    summaries of relevant code files. Do your best to generate an answer with the short summaries.  
    If you feel confident in your answer, respond with "The answer is:" followed by your answer.
    If you need more information, respond with "Need full context," and you will be prompted again with the 
    full relevant code file. 

    The user query is: How does the ad service work? 

    The summaries are: {'adservice.java': 'This JavaScript code sets up a gRPC server for a currency conversion service using the `@grpc/grpc-js` library. It includes functions for:\n- Initializing OpenTelemetry tracing (if enabled) and Stackdriver Profiler (if enabled).\n- Loading protocol buffer definitions and adding corresponding service definitions to the gRPC server.\n- Handling requests for supported currencies and currency conversions.\n- Handling health checks.\n- Starting the gRPC server on a

'Gemini responded with just the small summaries as context: The Ad service retrieves ads for a user, targeted based on the context provided in the request. If no context is provided, it serves random ads. It uses a Guava library for data structures and collections. The main class `AdService` starts and stops the gRPC server and blocks the main thread until server shutdown to prevent daemon threads from terminating the program prematurely. It also includes methods for retrieving ads by category or serving random ads if no category is specified. Additionally, it initializes OpenTelemetry stats and tracing (if enabled). The `AdServiceImpl` class implements the `AdServiceGrpc.AdServiceImplBase` interface, which defines the `getAds` method that processes ad requests and returns an `AdResponse` with a list of ads. 🎉'

In [131]:
# an example of a detailed query that requires the full code file (big)
small_to_big("exactly how does the SearchProducts() function work in the product catalog service?")


    You are a codebase helper. You will be given a user's question about the codebase, along with 
    summaries of relevant code files. Do your best to generate an answer with the short summaries.  
    If you feel confident in your answer, respond with "The answer is:" followed by your answer.
    If you need more information, respond with "Need full context," and you will be prompted again with the 
    full relevant code file. 

    The user query is: exactly how does the SearchProducts() function work in the product catalog service? 

    The summaries are: {'productcatalog.go': 'This JavaScript code sets up a gRPC server for a currency conversion service using the `@grpc/grpc-js` library. It includes functions for:\n- Initializing OpenTelemetry tracing (if enabled) and Stackdriver Profiler (if enabled).\n- Loading protocol buffer definitions and adding corresponding service definitions to the gRPC server.\n- Handling requests for supported currencies and currency conversions.\n-

'The `SearchProducts` function in the product catalog service takes a `SearchProductsRequest` that contains a query string and returns a `SearchProductsResponse` containing a list of products that match the query. The function first sleeps for a specified latency period to simulate network latency. It then iterates through the catalog of products and checks if the product name or description contains the query string in a case-insensitive manner. If a match is found, the product is added to the list of results. Finally, the function returns the list of results.'

In [ ]:
# the solution snippet of productcatalog.go ^ matches Gemini's response.  
"""
func (p *productCatalog) SearchProducts(ctx context.Context, req *pb.SearchProductsRequest) (*pb.SearchProductsResponse, error) {
	time.Sleep(extraLatency)

	var ps []*pb.Product
	for _, product := range p.parseCatalog() {
		if strings.Contains(strings.ToLower(product.Name), strings.ToLower(req.Query)) ||
			strings.Contains(strings.ToLower(product.Description), strings.ToLower(req.Query)) {
			ps = append(ps, product)
		}
	}

	return &pb.SearchProductsResponse{Results: ps}, nil
}
"""